In [1]:
%run /home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones_insercion_laboral.ipynb
#%run /Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones_insercion_laboral.ipynb
%run /home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/02_funciones_uso_del_tiempo_y_oportunidades.ipynb
#%run /Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/02_funciones_uso_del_tiempo_y_oportunidades.ipynb

import matplotlib.pyplot as plt
import pandas as pd
import warnings
import pickle
warnings.filterwarnings('ignore')

In [2]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathfolder_ENUT =  pathdata + 'data_input/ENUT_2021/Bases de datos/'

In [3]:
# Carga diccionarios

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerados.pickle', 'rb') as file:
    dict_cod_aglomerado = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia.pickle', 'rb') as file:
    dict_cod_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_provincia.pickle', 'rb') as file:
    map_aglomerado_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_region.pickle', 'rb') as file:
    map_aglomerado_region = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia_region.pickle', 'rb') as file:
    map_provincia_region = pickle.load(file)


dict_regiones_ENUT = {1: 'Gran Buenos Aires',
                    2: 'Pampeana',
                    3: 'Noroeste',
                    4: 'Noreste',
                    5: 'Cuyo',
                    6: 'Patagonia'}

In [4]:
# Carga CBT y adultos equivalentes

df_CBT = pd.read_csv(pathdata + 'data_output/Canasta_Basica_Total_Regiones_2016-2022-promedios-moviles.csv', delimiter=',', header=0, index_col=[0])
df_adultos_equiv = pd.read_csv(pathdata + 'data_input/canastas_basicas/adultos_equivalente.csv')

def capitalize_first_letter(s):
    return s.capitalize()

df_CBT = df_CBT.rename(columns=capitalize_first_letter)
df_CBT.rename(columns={'Gran_buenos_aires':'Gran Buenos Aires'}, inplace=True)

In [5]:
# Carga de bases EPH y ENUT

df_enut = pd.read_csv(pathfolder_ENUT + 'enut2021_base.txt', sep='|')
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

In [6]:
# Calculo de indicadores por componente

ratio_actividad, ratio_formalidad, ratio_jornada_laboral, ratio_ingreso_salarial = \
    indicadores_insercion_laboral(df_people_pool, dict_cod_provincia, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)

ratio_inactivos_con_ingreso, ratio_hogares_no_pobres_jefatura, ratio_jornada_no_paga = \
    indicadores_uso_del_tiempo_y_oportunidades(df_people_pool, df_houses_pool, df_enut, dict_regiones_ENUT, dict_cod_provincia, dict_cod_aglomerado, map_aglomerado_provincia, map_provincia_region, df_CBT, df_adultos_equiv, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)

In [7]:
# Ordeno indices de las componentes

ratio_actividad = ratio_actividad.sort_index()
ratio_formalidad = ratio_formalidad.sort_index()
ratio_jornada_laboral = ratio_jornada_laboral.sort_index()
ratio_ingreso_salarial = ratio_ingreso_salarial.sort_index()

ratio_jornada_no_paga = ratio_jornada_no_paga.sort_index()
ratio_inactivos_con_ingreso = ratio_inactivos_con_ingreso.sort_index()
ratio_hogares_no_pobres_jefatura = ratio_hogares_no_pobres_jefatura.sort_index()

In [8]:
# Exporto CSVs

ratio_actividad.to_csv(pathdata + 'data_output/indicadores/01_ratio_actividad.csv')
ratio_formalidad.to_csv(pathdata + 'data_output/indicadores/02_ratio_formalidad.csv')
ratio_jornada_laboral.to_csv(pathdata + 'data_output/indicadores/03_ratio_jornada_laboral.csv')
ratio_ingreso_salarial.to_csv(pathdata + 'data_output/indicadores/04_ratio_ingreso_salarial.csv')

ratio_jornada_no_paga.to_csv(pathdata + 'data_output/indicadores/05_ratio_jornada_no_paga.csv')
ratio_inactivos_con_ingreso.to_csv(pathdata + 'data_output/indicadores/06_ratio_inactivos_con_ingreso.csv')
ratio_hogares_no_pobres_jefatura.to_csv(pathdata + 'data_output/indicadores/07_ratio_hogares_no-pobres_jefatura.csv')